In [1]:
import h5py
import pandas as pd
import numpy as np

DATA_DIR = "../../data/logs"

In [2]:
with h5py.File(DATA_DIR + '/preprocessed_sm.h5','r') as hf:
    nextImages = hf['nextImage'][...]
    currentImages = hf['currentImage'][...]
    metadata = hf['metadata'][...]
    ids = hf['ids'][...]

In [3]:
df = pd.DataFrame([{ 
    "currentImage": ci,
    "nextImage": ni,
    "action": int(md[0]),
    "reward": md[1],
    "died": int(md[2]),
    "didBoost": int(md[3]),
    "id": i.decode()} for ni, ci, md, i in zip(nextImages, currentImages, metadata, ids)])

In [4]:
df = df.loc[(df['id'] != "00000000-0000-0000-0000-000000000000")]

In [5]:
import os
os.chdir('..')

In [6]:
from src.game import Trainer

In [7]:
mdl = Trainer.train_override(df.to_dict("records"),verbose=1)

1/1 [==============================] - 1s 519ms/step - loss: 1643185.8750 - move_loss: 9570.5332 - boost_loss: 1633615.3750 - move_accuracy: 0.6606 - boost_accuracy: 0.6157


In [8]:
import matplotlib.pyplot as plt

loss = mdl.history["loss"]
macc = mdl.history["move_accuracy"]
bacc = mdl.history["boost_accuracy"]
print(loss,macc,bacc)

[1643185.875] [0.66064453125] [0.61572265625]
